# THE OFFICIAL COLAB NOTEBOOK OF TEAM ABC FOR AUTOMIN @ INTERSPEECH 2021 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd drive/MyDrive/AutoMin
# !pip install -r requirements.txt

Mounted at /content/drive
/content/drive/MyDrive/AutoMin
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 48.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00


In [4]:
PREPROCESSED_DIR = "preprocessed_data"
OUTPUT_DIR = "minutes"

TRAIN_DIR = "train"
DEV_DIR = "dev"
TEST_DIR = "test"
TEST2_DIR = "test2"

MODEL_PATH = "models/bart_large_xsum_samsum"
MODEL_NAME = "facebook/bart-large-xsum"
CHECKPOINT = "checkpoint-5500"
PRETRAIN_DATASET = "samsum"
METRIC = "rouge"

In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name())

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# torch.cuda.set_device(0)

No GPU available, using the CPU instead.


In [7]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
import nltk
import re
import datetime
import json
import os

nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
summarizer = pipeline("summarization", model=os.path.join(MODEL_PATH, CHECKPOINT), device=0)

[nltk_data] Downloading package punkt to /Users/michelle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


OSError: Can't load the configuration of 'models/bart_large_xsum_samsum/checkpoint-5500'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'models/bart_large_xsum_samsum/checkpoint-5500' is the correct path to a directory containing a config.json file

In [4]:
def load_preprocessed_transcripts(file_name):
  with open(f"{file_name}.json", "r") as f:
    preprocessed_transcripts = json.load(f)

  return preprocessed_transcripts

In [5]:
cs_train_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "cs", TRAIN_DIR))
cs_dev_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "cs", DEV_DIR))
cs_test_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "cs", TEST_DIR))
cs_test2_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "cs", TEST2_DIR))

en_train_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "en", TRAIN_DIR))
en_dev_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "en", DEV_DIR))
en_test_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "en", TEST_DIR))
en_test2_preprocessed = load_preprocessed_transcripts(os.path.join(PREPROCESSED_DIR, "en", TEST2_DIR))

In [6]:
### CUSTOMIZED STRIP, REPLACE AND PREPROCESS FUNCTIONS ###

def stripp(string):
    list1=[]
    string = string.strip()
    list1[:0]=string
    idx = 0
    cnd = False
    for i in list1:
        if i.isalpha():
            cnd = True
            break
    if cnd:
        while list1[0].isalpha() == False:
            if idx+1 == len(string):
                break
            list1.remove(list1[0])
            idx+=1
        list1 = ''.join(list1)
    else:
        list1 = None

    return list1

def preprocess(ctx):
  ctx = ctx.replace(" '", "'")
  ctx = ctx.replace(" ,", ",")
  ctx = ctx.replace(" .", ".")
  ctx = ctx.replace(" ?", "?")
  ctx = ctx.replace("Ehmm", "")
  ctx = ctx.replace(" Ehm", "")
  ctx = ctx.replace(" mmm", "")
  ctx = ctx.replace(" hmm", "")
  ctx = ctx.replace(" uh", "")
  ctx = ctx.replace(" uh ,", "")
  ctx = ctx.replace(" uh .", "")
  ctx = ctx.replace(" um", "")
  ctx = ctx.replace(" um ,", "")
  ctx = ctx.replace(" um .", "")
  ctx = ctx.replace(" Uh", "")
  ctx = ctx.replace(" Uh ,", "")
  ctx = ctx.replace(" Uh .", "")
  ctx = ctx.replace(" Um", "")
  ctx = ctx.replace(" Um ,", "")
  ctx = ctx.replace(" Um .", "")
  ctx = ctx.replace("Uh", "")
  ctx = ctx.replace("Um", "")
  ctx = ctx.replace("Yeah", "")
  ctx = ctx.replace(" yeah", "")
  ctx = ctx.replace("Ehm, ", "")
  ctx = ctx.replace("Hmm, ", "")
  ctx = ctx.replace("Ehm. ", "")
  ctx = ctx.replace("Hmm. ", "")
  ctx = ctx.replace("Yeah", "")
  ctx = ctx.replace(" yeah", "")
  ctx = ctx.replace("Ehm", "")
  ctx = ctx.replace("Hmm", "")
  ctx = ctx.replace("Ehm", "")
  ctx = ctx.replace("Hmm", "")
  ctx = ctx.replace("Mhm", "")
  ctx = ctx.replace(" {disfmarker}", "")
  ctx = ctx.replace(" {vocalsound}", "")
  ctx = ctx.replace(" {gap}", "")
  ctx = ctx.replace("...", ".")
  ctx = ctx.replace("..", ".")
  ctx = ctx.replace(",,", ",")
  ctx = ctx.replace(",,", ",")
  ctx = ctx.replace(",.", "")
  ctx = ctx.replace(".,", ".")
  ctx = ctx.replace("  ", " ")
  ctx = ctx.replace("(", "")
  ctx = ctx.replace(")", "")
  ctx = ctx.replace("Person", "PERSON")
  ctx = ctx.replace("is going to", "will")
  ctx = ctx.replace("are going to", "will")
  ctx = ctx.replace("are discussing", "discussed")
  ctx = ctx.replace("discuss", "discussed")
  ctx = ctx.replace("are working", "worked")
  ctx = ctx.replace("is working", "worked")

  return ctx

def replacee(i):
  i = i.replace("do n't", "do not")
  i = i.replace("n't", "not")
  i = i.replace("it 's", "it is")
  i = i.replace(" 's", "")
  if i[0]+i[1] == "'s":
    i = i.replace("'s ", "")
  i = i.replace("wo n't", "won't")
  i = i.replace(" and", ",")
  i = i.replace(",,", ",")
  return i

def gen_tscs(length, transcripts, meeting_id=None):
  def split_line(line):
    splits = []

    line_ = line.split('.')
    split_ = len(line_)//2
    line1 = '. '.join(line_[0:split_]) + '.\n'
    line2 = role + ': ' + '. '.join(line_[split_:])

    for line in [line1, line2]:
      if len(tokenizer.encode(line)) >= length:
        splits += split_line(line)
      else:
        splits.append(line)

    return splits


  tscs_preprocessed = {}
  attendees = []

  for m_id, transcript in transcripts.items():
    if meeting_id is not None and m_id != meeting_id:
      continue

    roles = transcript['roles']
    attendees.append(sorted(list(set(roles))))
    utterances = transcript['utterances']
    tsc = ['']
    i=0
    for role, utterance in zip(roles, utterances):
      utterance = preprocess(utterance)
      v = re.sub(r"[^a-zA-Z0-9]+", ' ', utterance)
      v = v.split(' ')
      if len(v)<=4:
        continue
      if len(v)>4 and len(v)<7 and 's' in v:
        continue
      utterance = stripp(utterance)
      if utterance == None:
        continue
      if len(utterance) == 1:
        continue
      line = role + ': ' + utterance + '\n'

      # IF DIALOGUE IS LONGER THAN "length"
      tokenized_line = tokenizer.encode(line)
      if len(tokenized_line)>=length:
          line_splits = split_line(line)
      else:
          line_splits = [line]

      for l in line_splits:
          tokenized = tokenizer.encode(tsc[i]+l)
          if len(tokenized)>=length:
              i+=1
              tsc.append('')
              tsc[i]+=l
          else:
              tsc[i]+=l

    tscs = {m_id: tsc}
    tscs_preprocessed.update(tscs)

  return tscs_preprocessed, attendees

In [30]:
### IF INFERENCING ON A SPECIFIC TRANSCRIPT, INPUT THE MEETING ID... ###
m_id = 'meeting_en_test_006'

### IMPLEMENTING THE BELOW LINES WILL GIVE 3 SUMMARIES WITH VARYING LENGTHS, AS MENTIONED ###
tscs_preprocessed_long, attendees = gen_tscs(512, en_test_preprocessed, meeting_id=m_id) #for longer summary
tscs_preprocessed_avg, attendees = gen_tscs(768, en_test_preprocessed, meeting_id=m_id)
tscs_preprocessed_short, attendees = gen_tscs(1024, en_test_preprocessed, meeting_id=m_id) #for shorter summary

print(tscs_preprocessed_short[m_id][0])

PERSON11: Hi, how are you? Good morning.
PERSON4: Good morning. I'm, well, fine. Still at home.
PERSON4: You know. And you?
PERSON11: I'm also at home. But, ehm, the Czech republic government, they already lifted the Kind of lifted the rules. So, ehm, from this Monday we can actually go out even if it's not like the necessity. It the meetings of. Up to 10 people are allowed.
PERSON4: Ha. So here in LOCATION1, we have to wait until the 4th of May, some commercial activity can already be open. But we think that we have to wait until June for the free circulation of people. And fortunately, starting from the 4th of June we are allowed to reach our family. If it's in the same region.
PERSON4: And so finally, I will reach my parents. Cause we live in two different cities. Because I'm in Trento and my family is in Bolzano. Which is pretty near around 50 kilometres. But, but
PERSON11: So you're looking forward.
PERSON4: exactly. Hi guys, good morning.
PERSON11: So I went to the park yestreday

In [31]:
print(len(tscs_preprocessed_short[m_id]))
print(len(tscs_preprocessed_avg[m_id]))
print(len(tscs_preprocessed_long[m_id]))

11
14
21


In [32]:
# OVERVIEW THE SECTIONED BLOCKS OF CONVERSATIONS FROM THE TRANSCRIPT ...
for idx, i in enumerate(tscs_preprocessed_short[m_id]):
  print(f"{idx} - {i}")

0 - PERSON11: Hi, how are you? Good morning.
PERSON4: Good morning. I'm, well, fine. Still at home.
PERSON4: You know. And you?
PERSON11: I'm also at home. But, ehm, the Czech republic government, they already lifted the Kind of lifted the rules. So, ehm, from this Monday we can actually go out even if it's not like the necessity. It the meetings of. Up to 10 people are allowed.
PERSON4: Ha. So here in LOCATION1, we have to wait until the 4th of May, some commercial activity can already be open. But we think that we have to wait until June for the free circulation of people. And fortunately, starting from the 4th of June we are allowed to reach our family. If it's in the same region.
PERSON4: And so finally, I will reach my parents. Cause we live in two different cities. Because I'm in Trento and my family is in Bolzano. Which is pretty near around 50 kilometres. But, but
PERSON11: So you're looking forward.
PERSON4: exactly. Hi guys, good morning.
PERSON11: So I went to the park yestr

In [33]:
### USEFUL UTIL FUNCTIONS FOR GENERATION AND FORMATTING ###
def summarize(tsc):
  a1 = summarizer(tsc)[0]['summary_text']
  return a1

def format_summary_with_pronouns(s2):
  s3 = ''.join(s2) #s2[0]

  s3 = s3.split('.')
  summ = ['']
  id=0
  summ1 = []
  for i in s3:
    #stripping the spaces
    i = i.replace('  ', ' ')
    if len(i) == 1:
      continue
    if i[0]==' ' and i[1].isalpha():
      i = stripp(i)
    if type(i) == type(None):
      continue
    if i[0] == ' ':
      continue
    i = preprocess(i)
    check = re.sub(r"[^a-zA-Z0-9]+", ' ', i)
    check = ''.join(i for i in check if not i.isdigit())
    check = check.replace('  ', ' ')
    check = check.split(' ')
    if len(check)<=6:
      continue

    #formatting
    if i[0] == 'P' and i[1] == 'E':
      summ1.append('-' + i + '.')
    # elif i[0] in ['M','T','O','A'] and (i[1].isalpha()==False):
    #   id+=1
    #   summ.append('')
    #   summ[id] = summ[id] + ' -' + i + '.'
    # elif i[0]=='M' and i[1]=='U':
    #   id+=1
    #   summ.append('')
    #   summ[id] = summ[id] + ' -' + i + '.'
    else:
      summ1.append(i + '.')

  summ1 = insert_pronouns(summ1)
  for i in summ1:
    if i[1] == 'P' and i[2] == 'E':
      id+=1
      summ.append('')
      summ[id] = summ[id] + ' ' + i
    else:
      summ[id] = summ[id] + '\n  ' + i

  if '' in summ:
    summ.remove('')
  summ = '\n'.join(summ)
  return summ

### A FORMAT SUMMARY FUNCTION, WITHOUT PRONOUN INSERTION ###
def format_summary_without_pronouns(s2):
  s3 = ''.join(s2) #s2[0]

  s3 = s3.split('.')
  summ = ['']
  id=0

  for i in s3:
    #stripping the spaces
    i = i.replace('  ', ' ')
    if len(i) == 1:
      continue
    if i[0]==' ' and i[1].isalpha():
      i = stripp(i)
    if type(i) == type(None):
      continue
    if i[0] == ' ':
      continue
    i = preprocess(i)
    check = re.sub(r"[^a-zA-Z0-9]+", ' ', i)
    check = ''.join(i for i in check if not i.isdigit())
    check = check.replace('  ', ' ')
    check = check.split(' ')
    if len(check)<=6:
      continue

    #formatting
    if i[0] == 'P' and i[1] == 'E':
      id+=1
      summ.append('')
      summ[id] = summ[id] + ' -' + i + '.'
    # elif i[0] in ['M','T','O','A'] and (i[1].isalpha()==False):
    #   id+=1
    #   summ.append('')
    #   summ[id] = summ[id] + ' -' + i + '.'
    # elif i[0]=='M' and i[1]=='U':
    #   id+=1
    #   summ.append('')
    #   summ[id] = summ[id] + ' -' + i + '.'
    else:
      summ[id] = summ[id] + '\n  ' + i + '.'

  if '' in summ:
    summ.remove('')
  summ = '\n'.join(summ)
  return summ

def insert_pronouns(summ1):
  len_sum = len(summ1)
  for line_no, i in enumerate(summ1):
    if '-' in i:
      if len_sum-line_no <= 3:
        rng = len_sum-line_no-1
      else:
        rng = 3
      for k1 in range(rng):
        st1, st2 = check_req(i, summ1[line_no+k1+1])
        if st1:
          summ1[line_no+k1+1] = summ1[line_no+k1+1].replace(st1, 'They')
          summ1[line_no+k1+1] = summ1[line_no+k1+1].replace("They's", 'Their')
          summ1[line_no+k1+1] = summ1[line_no+k1+1].replace("They is", 'They are')
          summ1[line_no+k1+1] = summ1[line_no+k1+1].replace("They is", 'They are')
          summ1[line_no+k1+1] = summ1[line_no+k1+1].replace("They has", 'They have')
          summ1[line_no+k1+1] = summ1[line_no+k1+1].replace("They wants", 'They want')
  return summ1

def check_req(line1, line2):
  if ('-' in line1) and ('-' in line2):
    st1 = ''
    st2 = ''
    for _ in range(8):
      st1+=line1[_]
      st2+=line2[_]
    if st1 == st2:
      if line1[_+1] == line2[_+1]:
        if line1[_+1]==' ':
          st3 = st1
          st4 = st2
        elif line1[_+1]==',':
          st3 = False
          st4 = False
        else:
          st3 = st1+line1[_+1]
          st4 = st2+line2[_+1]
      else:
        if line1[_+1]=="'" or line2[_+1]=="'":
          st3 = st1
          st4 = st2
        else:
          st3 = False
          st4 = False
    else:
      st3 = False
      st4 = False
  else:
    st3 = False
    st4 = False

  return st3, st4

def gen_summaries(tscs_preprocessed):
  summaries = {}

  for k, v in tscs_preprocessed.items():
    if len(v) < 11:
      section = 2
    elif len(v) < 18:
      section = 4
    elif len(v) < 24:
      section = 6
    else:
      section = 8
    s1 = ['']
    tsc = v
    id=0
    for i, t1 in enumerate(tsc):
      a1 = summarize(t1)
      s1[id] = s1[id] + a1 + ' '
      if i%section==0:
        s1.append('')
        id+=1

    summaries[k] = s1

  return summaries

In [39]:
### THE BELOW 4 CELLS WOULD GIVE YOU 4 SUMMARIES VARYING IN LENGTH; ###
### THIS WOULD NORMALLY AFFECT THE COVERAGE AND ADEQUACY OF THE SUMMARIES; ###
### YOU CAN CHOOSE A SUITABLE SUMMARY FOR EVERY SINGLE TRANSCRIPT !!! ###

s2_short = gen_summaries(tscs_preprocessed_short)
print(format_summary_without_pronouns(s2_short[m_id]))

Your max_length is set to 62, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)



  The Czech Republic government has lifted the rules.
  People can go out even if they don't need to, but they have to wait until June for the free circulation of people.
 -PERSON4 lives in Trento and his family lives in Bolz PERSON1, PERSON8, PERSON11, PERSON13 and PERSON13 will write a project management guide for Organization 5.
  Organization 5 is developing their own systems in this area.
  Organization 3 is doing the dissemination on the web.
  Organization 1 is not keen on reading the guide PERSON11 will write something and someone will review it.
  The project management guides should be ready by the end of June at the latest and the internal reviews should be tested by mid June, so that they have two weeks to finish that.
 -PERSON8 has started the PROJECT1 test sets PERSON8 explains to PERSON1 and PERSON2 the layout of the test sets.
 -PERSON1, PERSON8, PERSON15 and PERSON13 worked on a project to develop a test set for spoken English translation.
  The test set should be lim

In [40]:
s2_avg = gen_summaries(tscs_preprocessed_avg)
print(format_summary_without_pronouns(s2_avg[m_id]))


  The Czech Republic government has lifted the rules.
  They can go out even if they don't need to, but they have to wait until June for the free circulation of people.
 -PERSON4 lives in Trento and his family lives in PERSON11, PERSON7, PERSON1, PERSON8, PERSON13 and PERSON16 will postpone one of their regular meetings to September.
  They have a couple of deliverables due in June.
  They haven't decided who will be their internal review person for any of them.
 -PERSON11 is writing a project management guide for Organization 3 and Organization 1.
  There is no description of the deliverable and there are no project management guides.
  Organization 1 has coordinated a number of projects.
  Organization 3 has period reports and data management plans.
  Organization 2 has project reports and PERSON11 will write something and someone will review it.
  The internal reviews should be ready by mid June at the latest.
  The deliverables will be due in the end of August.
  The Congress will

In [41]:
s2_long = gen_summaries(tscs_preprocessed_long)
print(format_summary_without_pronouns(s2_long[m_id]))


  The Czech Republic government has lifted the rules.
  People can go out even if they don't need to.
  People in LOCATION1 are allowed to meet up to 10 people.
  People living in Trento and Bolzano are able to meet their families starting PERSON11 will postpone one of their regular meetings in September.
  There will be two more meetings in May and one in June.
  There are six deliverables due in June, one of them technical.
  Organized 1 is not providing ASR system, so it will be good PERSON16 will do the summarization of the review.
 -PERSON8, PERSON1, PERSON11, PERSON13 and PERSON3 will be the reviewers.
 -PERSON11 is writing a project management guide for a new project.
  There is no description of the deliverable and there is no czech management guide.
  Organization 1 is in the best position to write it as they have coordinated a number of projects.
  Organization 2 is not keen on reading PERSON1 will write something and someone will review it.
  The internal reviews should be 

In [45]:
s2_long[m_id][1]

"PERSON11 is going to postpone one of their regular meetings in September. There will be two more meetings in May and one in June. There are six deliverables due in June, one of them technical. Organized 1 is not providing ASR system, so it will be good PERSON16 will do the summarization of the review. PERSON8, PERSON1, PERSON11, PERSON13 and PERSON3 will be the reviewers.  PERSON11 is writing a project management guide for a new project. There is no description of the deliverable and there is no czech management guide. Organization 1 is in the best position to write it as they have coordinated a number of projects. Organization 2 is not keen on reading PERSON1 will write something and someone will review it. The internal reviews should be ready by mid June at the latest, so that they have two weeks to finish that. QT21 doesn't have the project management guides. They have this period reports and data management plans, but PERSON8 and PERSON9 are going to discuss the delivery of the PR

# If we want to further shorten the obtained summary...
This method sacrifices gramaticality and readbility, in order to achieve compactness, by using NLTK stopword reduction over a general BART Summarization.

In [46]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kristyna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kristyna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
# RUN THE CELLS BELOW AND USE THIS FUNCTION INSTEAD OF THE 'format_summary()' version ...

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def shorten(example_sent):
  stop_words = set(stopwords.words('english'))
  stop_words.remove('to')
  stop_words.remove('of')
  stop_words.remove('from')
  stop_words.remove('as')
  stop_words.remove('has')
  stop_words.remove('do')
  stop_words.remove('not')
  #stop_words.remove('be')
  stop_words.remove('on')
  stop_words.remove('in')
  stop_words.remove('if')
  stop_words.remove('is')
  stop_words.remove('it')
  stop_words.remove('for')
  stop_words.remove('with')
  stop_words.remove('he')
  stop_words.remove('can')
  stop_words.remove('does')
  stop_words.remove('between')
  stop_words.add('They')
  stop_words.add('which')
  stop_words.add('On')
  stop_words.add('It')
  stop_words.add('The')
  stop_words.remove('over')
  stop_words.remove('until')
  stop_words.remove('after')
  stop_words.add('He')
  stop_words.remove('when')
  stop_words.remove('have')
  stop_words.remove('them')
  stop_words.remove('into')
  stop_words.remove('by')
  stop_words.remove('and')
  stop_words.remove('will')
  stop_words.remove('what')
  stop_words.add('manually')
  stop_words.remove('him')

  word_tokens = word_tokenize(example_sent)

  filtered_sentence = [w for w in word_tokens if w not in stop_words]

  return ' '.join(filtered_sentence)

def format_summary_short(s2):
  s3 = ''.join(s2) #s2[0]
  s3 = s3.split('.')
  summ = ['']
  id=0
  for i in s3:

    #stripping the spaces
    i = i.replace('  ', ' ')
    if len(i) == 1:
      continue
    if i[0]==' ' and i[1].isalpha():
      i = stripp(i)
    if i[0] == ' ':
      continue
    check = re.sub(r"[^a-zA-Z0-9]+", ' ', i)
    check = ''.join(i for i in check if not i.isdigit())
    check = check.replace('  ', ' ')
    check = check.split(' ')
    if len(check)<=6:
      continue

    #formatting
    if i[0] == 'P':
      id+=1
      summ.append('')
      i = shorten(i)
      i = replacee(i)
      summ[id] = summ[id] + ' -' + i + '.'
    else:
      i = shorten(i)
      i = replacee(i)
      summ[id] = summ[id] + '\n  ' + i + '.'

  if '' in summ:
    summ.remove('')
  summ = '\n'.join(summ)
  return summ

In [48]:
from transformers import BartTokenizer, BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

In [50]:
print(format_summary_short(s2_short[m_id]))


  Czech Republic government has lifted rules.
 -People can go even if do not need to , have to wait until June for free circulation of people.
 -PERSON4 lives in Trento, family lives in Bolz PERSON1 , PERSON8 , PERSON11 , PERSON13, PERSON13 going to write project management guide for Organization 5.
  Organization 5 is developing systems in area.
  Organization 3 is dissemination on web.
  Organization 1 is not keen on reading guide PERSON11 will write something, someone will review it.
  project management guides ready by end of June latest, internal reviews tested by mid June , have two weeks to finish.
 -PERSON8 has started PROJECT1 test sets PERSON8 explains to PERSON1, PERSON2 layout of test sets.
 -PERSON1 , PERSON8 , PERSON15, PERSON13 working on project to develop test set for spoken English translation.
  test set limited to 3 file lists for general public.
 -PERSON15 has to send source code of task to Organization 1.
 -Person8 asks Organization 1 to review PROJECT1 test set,

In [57]:
summary = ''
for s in s2_short[m_id]:
  preprocess_text = s.strip().replace("\n","")
  inputs = tokenizer(preprocess_text, return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=512)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
  summary = summary + output + ' '

In [58]:
summary

"The Czech Republic government has lifted the rules. People can go out even if they don't need to, but they have to wait until June for the free circulation of people. PERSON4 lives in Trento and his family lives in Bolz. PERSON11 went to the park yesterday. PERSON1, PERSON8, PERSON11, PERSON13 and PERSON13 are going to write a project management guide for Organization 5. Organization 5 is developing their own systems in this area. The project management guides should be ready by the end of June at the latest and the internal reviews should be tested by mid June. PERSON15 has to send the source code of the task to the Organization 1. Person8 asks Organization 1 to review the PROJECT1 test set and contribute to it. The date for the demo is 14th May 17th. The project was started when the EU still existed, so they will see what happened. PERSON4, PERSON8, PERSON1, PERSON15 and PERSON8 discuss how to present the results of the demo to the project officer. The demo should be in German. PERS

In [21]:
# Generate minutes
def format_minutes(attendees, minutes):
  tday = datetime.date.today()
  att = ", ".join(attendees[0])
  return f"DATE : {tday}\nATTENDEES : {att}\n\n\nSUMMARY-\n{minutes}\n\n\nMinuted by: Team ABC"

def generate_minutes(preprocessed_transcripts, output_dir):
  for length in [512, 768, 1024]:
    split_transcripts, attendees = gen_tscs(length, preprocessed_transcripts)
    summaries = gen_summaries(split_transcripts)

    for meeting_id, summary in summaries.items():
      minutes_with_pronouns = format_summary_with_pronouns(summary)
      minutes_without_pronouns = format_summary_without_pronouns(summary)
      minutes_shorten = format_summary_short(summary)

      os.makedirs(os.path.join(output_dir, meeting_id), exist_ok=True)

      with open(os.path.join(output_dir, meeting_id, f"length_{length}_with_pronouns"), "w") as f:
        f.write(format_minutes(attendees, minutes_with_pronouns))

      with open(os.path.join(output_dir, meeting_id, f"length_{length}_without_pronouns"), "w") as f:
        f.write(format_minutes(attendees, minutes_without_pronouns))

      with open(os.path.join(output_dir, meeting_id, f"length_{length}_shorten"), "w") as f:
        f.write(format_minutes(attendees, minutes_shorten))

In [22]:
# generate_minutes(cs_train_preprocessed, os.path.join(OUTPUT_DIR, "cs", TRAIN_DIR))
# generate_minutes(cs_dev_preprocessed, os.path.join(OUTPUT_DIR, "cs", DEV_DIR))
# generate_minutes(cs_test_preprocessed, os.path.join(OUTPUT_DIR, "cs", TEST_DIR))
# generate_minutes(cs_test2_preprocessed, os.path.join(OUTPUT_DIR, "cs", TEST2_DIR))

generate_minutes(en_train_preprocessed, os.path.join(OUTPUT_DIR, "en", TRAIN_DIR))
generate_minutes(en_dev_preprocessed, os.path.join(OUTPUT_DIR, "en", DEV_DIR))
generate_minutes(en_test_preprocessed, os.path.join(OUTPUT_DIR, "en", TEST_DIR))
generate_minutes(en_test2_preprocessed, os.path.join(OUTPUT_DIR, "en", TEST2_DIR))

Token indices sequence length is longer than the specified maximum sequence length for this model (2759 > 1024). Running this sequence through the model will result in indexing errors
Your max_length is set to 62, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 62, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 62, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 62, but you input_length is only 36. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 62, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 62, but you input_length is only 3. Yo

# TextRank Scipt for ranking sentences
This method uses GloVe Embeddings to calculate similarity score with the help of cosine similairty, and ranks individual sentences with the help of the PageRank Algorithm.

In [59]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /home/kristyna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [61]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip -d models/glove
!rm -rf glove*.zip

glove.6B.zip        100%[===================>] 822.24M   633KB/s    in 13m 1s  

2023-04-27 10:29:52 (1.05 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Archive:  glove.6B.zip
  inflating: models/glove.6B.50d.txt  
  inflating: models/glove.6B.100d.txt  
  inflating: models/glove.6B.200d.txt  
  inflating: models/glove.6B.300d.txt  
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2023-04-2

In [63]:
# ENTER THE MINUTE_ID
min_id = 'meeting_en_train_001'

import os
# path = 'outputs'
path = OUTPUT_DIR + '/en/train'
# os.chdir(path)
summaries = []
# for file1 in sorted(os.listdir()):
sumfile = open(path + '/' + min_id + '/' + 'length_1024_with_pronouns', 'r')
summ = sumfile.readlines()
summ = summ[5:-3]
text = ''
for line in summ:
    line = line.replace(' -', '')
    line = line.replace('  ', '')
    line = line.replace('\n', '')
    text = text + line + ' '
summaries.append(text)
#break

In [65]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in summaries:
  sentences.append(sent_tokenize(s))

sentences = [(idx, y) for x in sentences for idx, y in enumerate(x)] # flatten list
print('Total no. of sentences: ', len(sentences))

Total no. of sentences:  22


In [67]:
# EXTRACT WORD VECTORS

word_embeddings = {}
f = open('models/glove/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()


# REMOVE PUNCTUATIONS, NUMBERS AND SPECIAL CHARACTERS
# clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = pd.DataFrame(sentences, columns = ['order', 'sentence'])


# MAKE ALPHABETS TO LOWERCASE
# clean_sentences = [s.lower() for s in clean_sentences]
clean_sentences['sentence'] = clean_sentences['sentence'].str.replace("[^a-zA-Z]", " ").str.lower()

# REMOVE STOPWORDS

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences['sentence'] = clean_sentences['sentence'].apply(lambda x: remove_stopwords(x.split()))

# EXTRACT SENTENCE VECTORS

sentence_vectors = []
# for i in clean_sentences:
def get_sentence_vectors(row):
  sentence_vector = None
  sentence = row['sentence']
  if len(sentence) != 0:
    sentence_vector = sum([word_embeddings.get(w, np.zeros((100,))) for w in sentence.split()])/(len(sentence.split())+0.001)
  else:
    sentence_vector = np.zeros((100,))
  return sentence_vector

clean_sentences['sentence_vector'] = clean_sentences.apply(lambda x: get_sentence_vectors(x), axis=1)


# INITIALIZE A SIMILARITY MATRIX
sim_mat = np.zeros([len(sentences), len(sentences)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      # sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
      sim_mat[i][j] = cosine_similarity(clean_sentences['sentence_vector'][i].reshape(1,100), clean_sentences['sentence_vector'][j].reshape(1,100))[0,0]


# PAGERANK SCORING

import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kristyna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
# REVIEW THE RANKINGS BEFORE ELIMINATING IRRELEVANT INFO FROM THE SUMMARY

for ranked_sentence in ranked_sentences:
  print(ranked_sentence)

(0.051023536570241404, 'The project management guides should be ready by the end of June at the latest and the internal reviews should be tested by mid June, so that they have two weeks to finish that.')
(0.05097876245559428, 'PERSON8 asks Organization 1 to review the PROJECT1 test set and contribute to it.')
(0.0506864561085652, 'The project was started when the EU still existed, so they will see what happened.')
(0.0502904519060587, 'Organization 1 is not keen on reading the guide PERSON11 will write something and someone will review it.')
(0.049871040776940094, 'Organization 5 PERSON4, PERSON8, PERSON1, PERSON15 and PERSON8 discussed how to present the results of the demo to the project officer.')
(0.04980569948654663, "People can go out even if they don't need to, but they have to wait until June for the free circulation of people.")
(0.04941230787880488, 'PERSON1, PERSON8, PERSON15 and PERSON13 worked on a project to develop a test set for spoken English translation.')
(0.04881540

In [70]:
# ENTER THE PERCENTAGE OF SENTENCES THAT SEEM UNIFORMATIONAL,  THIS NUMBER IS USUALLY AROUND ~15% FOR THE MINUTES BELONGING TO A LENGTHY TRANSCRIPT

rem_perc = 0.15

import math
remove_count = math.ceil(len(sentences)*rem_perc)
print('No. of sentences removed: ', remove_count)

print('\n\nReduced Summary(jumbled): \n')
for i in range(len(ranked_sentences)-remove_count):
  print(ranked_sentences[i][1])

No. of sentences removed:  4


Reduced Summary(jumbled): 

The project management guides should be ready by the end of June at the latest and the internal reviews should be tested by mid June, so that they have two weeks to finish that.
PERSON8 asks Organization 1 to review the PROJECT1 test set and contribute to it.
The project was started when the EU still existed, so they will see what happened.
Organization 1 is not keen on reading the guide PERSON11 will write something and someone will review it.
Organization 5 PERSON4, PERSON8, PERSON1, PERSON15 and PERSON8 discussed how to present the results of the demo to the project officer.
People can go out even if they don't need to, but they have to wait until June for the free circulation of people.
PERSON1, PERSON8, PERSON15 and PERSON13 worked on a project to develop a test set for spoken English translation.
The test set should be limited to 3 file lists for the general public.
Organization 5 is developing their own systems in this a

In [ ]:
ranked_sentences.sort(key=lambda sentence: sentence[1][0])

for sentence in ranked_sentences:
  print(sentence[1][1])